
## Profiling three methods of iteration

In this note book, I use profilling to find the information of where the code to optimize are.

In [1]:
import random
import numpy as np
from numba import jit
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from time import time
import timeit
from functools import partial
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from codes.ddp_auxiliary import *
from codes.ddp_plots import *
from codes.ddp_algorithms import modified_policy_iteration
from codes.ddp_algorithms import policy_iteration
from codes.ddp_algorithms import value_iteration
from codes.ddp_algorithms import bellman_equation
from codes.ddp_algorithms import evaluate_policy

In [2]:
import cProfile, pstats, io

def profile(fnc):
    
    """
    A decorator that uses cProfile to profile a function
    source: https://osf.io/upav8/
    """
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [5]:
paras = {
    'beta': 0.95,
    'alpha': 0.3,
    'delta': 0.1,
    'sigma': 2,
}
capitals, consumptions, A = linearized_dynamic_system(**paras)

max_iter, num_states= 500, 400

# Create grid world
k_grid, c_grid, u_grid =  get_grid(dev = None, num_states = num_states, **paras)

### Value iteration

In [9]:
def bellman_equation(u_grid, val_old, beta):
    """
    Computes and returns the updated
    value function for an old value function `val_old`.
    Return NA position to 0 value

    Parameters
    ----------
    val_old : array_like(float, ndim=1)
        Old value function vector, of length n.

    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
        Utility grid.

    Returns
    -------
    v_new : array_like( 2-dimensional ndarray of shape (n, n))
        Updated utility value grid

    """
    v = u_grid + beta*val_old[:,None]

    v_new = np.where(np.isnan(u_grid), 0, v)

    return v_new

def state_wise_max(u_grid):
    """
    Find the maximum value function and the corresponding policy
    
    Parameters
    ----------
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    Returns
    -------
    value_fn : array_like(float, ndim=1)
            Value function vector, of length n.

    policy : ndarray(float, ndim=2)
        Transition matrix for `policy`, of shape (n, n).
    """
    u_new = np.where(np.isnan(u_grid), 0, u_grid)

    value_fn = np.amax(u_new, axis = 0)
    policy = np.argmax(u_new, axis = 0)
    return value_fn, policy

@profile
def value_iteration(max_iter, u_grid, beta,crit):
    """
    Solve the optimization problem by value iteration.

    """
    if crit is None:
        crit = 1e-06
    
    num_states = len(u_grid)

    # set up
    value_store_iter = np.tile(np.nan, [max_iter, num_states])
    store_policy = np.zeros((max_iter, num_states), dtype= int)
    val_old, val_new = np.zeros(num_states), np.zeros(num_states)

    for i in range(max_iter):
        v_new = bellman_equation(u_grid, val_old, beta)
        val_new, store_policy[i,:] = state_wise_max(v_new)
        
        max_diff = np.max(np.absolute(val_new - val_old))
        # quit iterations, when convergence is achieved
        if max_diff < crit:
            break

        value_store_iter[i,:] = val_new
        val_old = val_new

    num_iter = i

    return value_store_iter, store_policy, num_iter

value_iteration(max_iter, u_grid, paras["beta"], crit = None)

         6564 function calls in 0.595 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.053    0.053    0.595    0.595 <ipython-input-9-8ecabb5ab709>:50(value_iteration)
     1056    0.167    0.000    0.419    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      211    0.015    0.000    0.346    0.002 <ipython-input-9-8ecabb5ab709>:27(state_wise_max)
      211    0.107    0.001    0.192    0.001 <ipython-input-9-8ecabb5ab709>:1(bellman_equation)
      211    0.000    0.000    0.174    0.001 <__array_function__ internals>:2(argmax)
      211    0.000    0.000    0.174    0.001 C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:1112(argmax)
      211    0.000    0.000    0.174    0.001 C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:55(_wrapfunc)
      211    0.173    0.001    0.173    0.001 {method 'argmax' of 'numpy.ndarray' objects}
      

(array([[0.0378547 , 0.04047529, 0.04308127, ..., 0.53287905, 0.53347246,
         0.53406433],
        [0.07388349, 0.07653825, 0.07918412, ..., 0.7367631 , 0.73781653,
         0.73886715],
        [0.10823804, 0.1109255 , 0.11360273, ..., 0.85662704, 0.85795605,
         0.85928378],
        ...,
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]]),
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  2,   3,   3, ..., 203, 203, 204],
        [  4,   5,   6, ..., 270, 270, 271],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]),
 210)

In [18]:
@jit
def boosted_state_wise_max(u_grid):
    """
    Find the maximum value function and the corresponding policy
    
    Parameters
    ----------
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    Returns
    -------
    value_fn : array_like(float, ndim=1)
            Value function vector, of length n.

    policy : ndarray(float, ndim=2)
        Transition matrix for `policy`, of shape (n, n).
    """
    u_new = np.where(np.isnan(u_grid), 0, u_grid)

    value_fn = np.amax(u_new, axis = 0)
    policy = np.argmax(u_new, axis = 0)
    return value_fn, policy

def state_wise_max(u_grid):
    """
    Find the maximum value function and the corresponding policy
    
    Parameters
    ----------
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    Returns
    -------
    value_fn : array_like(float, ndim=1)
            Value function vector, of length n.

    policy : ndarray(float, ndim=2)
        Transition matrix for `policy`, of shape (n, n).
    """
    u_new = np.where(np.isnan(u_grid), 0, u_grid)

    value_fn = np.amax(u_new, axis = 0)
    policy = np.argmax(u_new, axis = 0)
    return value_fn, policy

In [19]:
%timeit boosted_state_wise_max(u_grid)

1.42 ms ± 36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit state_wise_max(u_grid)

1.37 ms ± 67.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
@jit
def boosted_bellman_equation(u_grid, val_old, beta):
    """
    Computes and returns the updated
    value function for an old value function `val_old`.
    Return NA position to 0 value

    Parameters
    ----------
    val_old : array_like(float, ndim=1)
        Old value function vector, of length n.

    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
        Utility grid.

    Returns
    -------
    v_new : array_like( 2-dimensional ndarray of shape (n, n))
        Updated utility value grid

    """
    v = u_grid + beta*val_old[:,None]

    v_new = np.where(np.isnan(u_grid), 0, v)

    return v_new

def bellman_equation(u_grid, val_old, beta):
    """
    Computes and returns the updated
    value function for an old value function `val_old`.
    Return NA position to 0 value

    Parameters
    ----------
    val_old : array_like(float, ndim=1)
        Old value function vector, of length n.

    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
        Utility grid.

    Returns
    -------
    v_new : array_like( 2-dimensional ndarray of shape (n, n))
        Updated utility value grid

    """
    v = u_grid + beta*val_old[:,None]

    v_new = np.where(np.isnan(u_grid), 0, v)

    return v_new

In [22]:
val_old= np.zeros(num_states)

In [23]:
%timeit boosted_bellman_equation(u_grid, val_old, paras["beta"])

1.01 ms ± 30.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit bellman_equation(u_grid, val_old, paras["beta"])

1.04 ms ± 40.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Policy iteration

In [20]:
def evaluate_policy(policy, u_grid, beta):
    """
    Computes the updated value function `Tv` for a `policy`.

    Parameters
    ----------
    policy : array_like(int, ndim=1)
        Policy vector, of length n.
        
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    U_policy : array_like(float, ndim=1)
            Utility vector corresponding with policy, of length n
    
    Q_policy : array_like( 2-dimensional ndarray of shape (n, n))
             sparse matrix Q with zeros everywhere except for its row i and column j elements

    Returns
    -------
    v_policy : array_like(float, ndim=1)
        Value function vector, of length n

    """
    num_states = len(u_grid)

    # Solve (I - beta * Q_policy) v = U_policy
    U_policy = u_grid[policy,range(len(policy))]
    b = U_policy

    Q_policy = np.zeros((num_states, num_states))
    Q_policy[range(len(policy)),policy] = 1.0
    
    I = np.identity(num_states)
    A = I - beta * Q_policy

    v_policy = np.linalg.solve(A, b)
    return v_policy

def v_greedy(v, u_grid, beta):
    """
    Parameters
    ----------
    v : array_like(float, ndim=1)
        Value function vector, of length n.

    policy : ndarray(int, ndim=1)
        Optional output array for `sigma`.
    Returns
    -------
    policy : ndarray(int, ndim=1)
        v-greedy policy vector, of length n.
    """

    #improve value function
    v_new = bellman_equation(u_grid, v, beta)
    
    #find the new_policy (new maximum)
    new_value , new_policy = state_wise_max(v_new)
    return new_value, new_policy

def random_policy(_grid):
    """
    Randomly choose a policy from plausible choices from grid world

    Parameters
    ----------
    c_grid : array_like( 2-dimensional ndarray of shape (n, n))
        Utility vector, of length n

    Returns
    -------
    policy : ndarray(int, ndim=1)
            random policy vector, of length n
    """
    new_grid = np.where(np.isnan(_grid), 0, _grid)

    a = np.transpose(new_grid)

    policy = [np.random.choice(r.nonzero()[0]) for r in a]

    return policy

@profile
def policy_iteration(max_iter, u_grid, beta):
    """
    Solve the optimization problem by policy iteration

    matrix Q with zeros everywhere except for its row i and column j elements, which equal one

    """

    num_states = len(u_grid)

    # set up
    value_store_iter = np.tile(np.nan, [max_iter, num_states])
    store_policy = np.tile(np.nan, [max_iter, num_states])

    # Initialize with a random policy and initial value function
    policy = random_policy(u_grid)
    v_policy = evaluate_policy(policy, u_grid, beta)

    for i in range(max_iter):
        # Policy improvement
        improved_value , improved_policy = v_greedy(v_policy, u_grid, beta)
    
        # Policy evaluation
        Tv = evaluate_policy(improved_policy, u_grid, beta)

        store_policy[i+1,:] = improved_policy
        value_store_iter[i+1,:] = improved_value

        # quit iterations, when convergence is achieved
        if np.array_equal(improved_policy, policy):
            break

        policy = improved_policy
        v_policy = Tv

    num_iter = i + 1

    return value_store_iter, store_policy, num_iter
    
policy_iteration(max_iter, u_grid, paras["beta"])

         6555 function calls in 0.124 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.008    0.008    0.124    0.124 <ipython-input-20-4bd3df77f247>:84(policy_iteration)
      101    0.014    0.000    0.065    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
       16    0.004    0.000    0.052    0.003 <ipython-input-20-4bd3df77f247>:40(v_greedy)
       17    0.022    0.001    0.052    0.003 <ipython-input-20-4bd3df77f247>:1(evaluate_policy)
       16    0.001    0.000    0.031    0.002 <ipython-input-18-9b6d17d4b666>:27(state_wise_max)
       17    0.000    0.000    0.025    0.001 <__array_function__ internals>:2(solve)
       17    0.025    0.001    0.025    0.001 C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:323(solve)
       16    0.009    0.001    0.017    0.001 <ipython-input-9-8ecabb5ab709>:1(bellman_equation)
       16    0.000    0.000    0.016    0

(array([[       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [0.32011984, 0.33022174, 0.34021487, ..., 1.77618784, 1.77791662,
         1.77963779],
        [0.70513693, 0.70958792, 0.7140068 , ..., 1.78626219, 1.78822214,
         1.79017975],
        ...,
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]]),
 array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [194., 194., 194., ..., 337., 337., 337.],
        [ 92.,  92.,  92., ..., 366., 366., 367.],
        ...,
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]]),
 16)

In [25]:
@jit
def boosted_evaluate_policy(policy, u_grid, beta):
    """
    Computes the updated value function `Tv` for a `policy`.

    Parameters
    ----------
    policy : array_like(int, ndim=1)
        Policy vector, of length n.
        
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    U_policy : array_like(float, ndim=1)
            Utility vector corresponding with policy, of length n
    
    Q_policy : array_like( 2-dimensional ndarray of shape (n, n))
             sparse matrix Q with zeros everywhere except for its row i and column j elements

    Returns
    -------
    v_policy : array_like(float, ndim=1)
        Value function vector, of length n

    """
    num_states = len(u_grid)

    # Solve (I - beta * Q_policy) v = U_policy
    U_policy = u_grid[policy,range(len(policy))]
    b = U_policy

    Q_policy = np.zeros((num_states, num_states))
    Q_policy[range(len(policy)),policy] = 1.0
    
    I = np.identity(num_states)
    A = I - beta * Q_policy

    v_policy = np.linalg.solve(A, b)
    return v_policy

def evaluate_policy(policy, u_grid, beta):
    """
    Computes the updated value function `Tv` for a `policy`.

    Parameters
    ----------
    policy : array_like(int, ndim=1)
        Policy vector, of length n.
        
    u_grid : array_like( 2-dimensional ndarray of shape (n, n))
            Utility grid

    U_policy : array_like(float, ndim=1)
            Utility vector corresponding with policy, of length n
    
    Q_policy : array_like( 2-dimensional ndarray of shape (n, n))
             sparse matrix Q with zeros everywhere except for its row i and column j elements

    Returns
    -------
    v_policy : array_like(float, ndim=1)
        Value function vector, of length n

    """
    num_states = len(u_grid)

    # Solve (I - beta * Q_policy) v = U_policy
    U_policy = u_grid[policy,range(len(policy))]
    b = U_policy

    Q_policy = np.zeros((num_states, num_states))
    Q_policy[range(len(policy)),policy] = 1.0
    
    I = np.identity(num_states)
    A = I - beta * Q_policy

    v_policy = np.linalg.solve(A, b)
    return v_policy


In [26]:
policy = random_policy(u_grid)

In [30]:
%timeit boosted_evaluate_policy(policy, u_grid, paras["beta"])

4.34 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%timeit evaluate_policy(policy, u_grid, paras["beta"])

4.13 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
